# First Test for EVT

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
from itertools import product

# Import functions
import backtesting as BT
import plots as IP
import evt as EVT

### Data

In [2]:
# Download data
sp500_data = yf.download("^GSPC", start="1981-01-01", end="2021-01-01")
sp500_data["Log Returns"] = np.log(sp500_data["Close"] / sp500_data["Close"].shift(1))
returns = sp500_data["Log Returns"].dropna()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


### EVT

In [3]:
# Set parameters
confidence_level = 0.99 # <----- Can choose 0.95 etc

In [4]:
# Apply EVT model
evt_output, var_estimate, es_estimate, prob_exceedance = EVT.evt(
    returns=returns,
    confidence_level=confidence_level,
    threshold_percentile=97,
    exceedance_level=0.05,  # optional: probability of exceeding 5% loss
    wealth=1000000
)

# Print VaR and ES Estimates
print(f"EVT VaR estimate $ (abs): {var_estimate:.2f}")
print(f"EVT Expected Shortfall estimate $ (abs): {es_estimate:.2f}")

# If you also want the probability of exceeding 5% loss:
if prob_exceedance is not None:
    print(f"Probability of exceeding 5% daily loss: {prob_exceedance * 100:.2f}%")

# Plot interactive VaR
fig_var = IP.plot_var(evt_output)

# Plot interactive ES for a subset
fig_es = IP.plot_es(evt_output, subset=("2017-11-01", "2020-11-30"), interactive=True)


EVT VaR estimate $ (abs): 31488.45
EVT Expected Shortfall estimate $ (abs): 48582.19
Probability of exceeding 5% daily loss: 0.27%


In [5]:
evt_output.head()

,Returns,VaR,ES,VaR Violation,VaR_monetary,ES_monetary
Date,,,,,,
1981-01-05,0.011885,0.031488,0.048582,False,31488.454539,48582.185273
1981-01-06,0.001087,0.031488,0.048582,False,31488.454539,48582.185273
1981-01-07,-0.022256,0.031488,0.048582,False,31488.454539,48582.185273
1981-01-08,-0.015067,0.031488,0.048582,False,31488.454539,48582.185273
1981-01-09,0.003151,0.031488,0.048582,False,31488.454539,48582.185273


In [6]:
evt_output, var_estimate, es_estimate, prob_exceedance, diag = EVT.evt(
    returns=returns,
    confidence_level=confidence_level,
    threshold_percentile=99,
    exceedance_level=0.05,
    diagnostics=True
)

print(f"xi: {diag['xi']:.4f}, beta: {diag['beta']:.4f}")
print(f"Threshold u: {diag['threshold_u']:.4f}")
print(f"Max support (x_max): {diag['max_support']:.4f}")
print(f"Number of exceedances: {diag['num_exceedances']}")


xi: 0.3299, beta: 0.0118
Threshold u: 0.0310
Max support (x_max): inf
Number of exceedances: 101


### Backtesting (Extensive)

In [7]:
total_violations, violation_rate = BT.count_violations(evt_output)

print(f"Total VaR Violations: {total_violations}")
print(f"Violation Rate: {violation_rate:.2f}%")


Total VaR Violations: 101
Violation Rate: 1.00%


In [8]:
kupiec_results = BT.kupiec_test(
    total_violations=total_violations,
    total_days=len(evt_output),
    confidence_level=confidence_level
)

print("\nKupiec Test Results:")
for key, value in kupiec_results.items():
    print(f"{key}: {value}")


Kupiec Test Results:
LR_uc: 0.00019620184798441187
p_value: 0.9888242312457648
reject_null: False


In [9]:
christoffersen_results = BT.christoffersen_test(evt_output)

print("\nChristoffersen Test Results:")
for key, value in christoffersen_results.items():
    print(f"{key}: {value}")



Christoffersen Test Results:
LR_c: 24.6855327447322
p_value: 6.748888944363429e-07
reject_null: True


In [10]:
joint_results = BT.joint_lr_test(
    LR_uc=kupiec_results["LR_uc"],
    LR_c=christoffersen_results["LR_c"]
)

print("\nJoint Test Results:")
for key, value in joint_results.items():
    print(f"{key}: {value}")


Joint Test Results:
LR_total: 24.685728946580184
p_value: 4.3607588251504126e-06
reject_null: True
